## 7-4 eval
### 7-4-1 eval族

In [1]:
eval 'p self'

main


main

In [2]:
object = Object.new
object.instance_eval {p self}

#<Object:0x0000555f471646e8>


#<Object:0x0000555f471646e8>

In [3]:
object.class.class_eval {p self}

Object


Object

In [4]:
Kernel.module_eval {p self}

Kernel


Kernel

### 7-4-2 Kernel.#eval

In [5]:
eval "1+1"

2

In [6]:
a = 10
eval "a = 1"
a

1

In [7]:
class LoggingInstanceVariable
    logging_instance_val_names = %w(first_val second_val third_val)
    
    logging_instance_val_names.each do |val_name|
        eval <<-EOS
            attr_accessor :#{val_name}
        EOS
    end
end

["first_val", "second_val", "third_val"]

In [8]:
obj = LoggingInstanceVariable.new

#<LoggingInstanceVariable:0x0000555f46990bd8>

In [9]:
obj.first_val = 1
obj.first_val

1

In [10]:
obj.second_val = 2
obj.second_val

2

In [11]:
obj.respond_to? 'third_val'

true

### 7-4-3 evalとBindingオブジェクト

In [12]:
p eval "@instane_val"
eval "local_val"

nil


NameError: undefined local variable or method `local_val' for main:Object
Did you mean?  local_variables

In [13]:
class EvalTarget
    def initialize
        @instance_val = 'instance valiable'
    end
    
    def instance_binding
        local_val = 'local valiable'
        binding
    end
    
    private
    def private_method
        'private method'
    end
end

:private_method

In [14]:
e1 = EvalTarget.new

#<EvalTarget:0x0000555f47218850 @instance_val="instance valiable">

In [15]:
binding_obj = e1.instance_binding

#<Binding:0x0000555f471ac470>

In [16]:
eval "@instance_val", binding_obj

"instance valiable"

In [17]:
eval "local_val", binding_obj

"local valiable"

In [18]:
eval "private_method", binding_obj

"private method"

In [19]:
binding_obj.eval "@instance_val"

"instance valiable"

In [20]:
binding_obj.eval "local_val"

"local valiable"

In [21]:
binding_obj.eval "private_method"

"private method"

### 7-4-4 module_eval / class_eval / instance_eval

In [22]:
class EvalTarget
    @class_class_instance = 10
    class << self
        def class_class_instance
            @class_class_instance
        end
    end
end

:class_class_instance

In [23]:
EvalTarget.class_class_instance

10

In [24]:
EvalTarget.class_eval do
    @class_class_instance = 5
end

5

In [25]:
EvalTarget.class_class_instance

5

In [26]:
val = 1

class DefineClass
    @class_class_instane = val
end

NameError: undefined local variable or method `val' for DefineClass:Class
Did you mean?  eval

In [27]:
class EvalTarget; end

val = 1

EvalTarget.class_eval do
    @class_class_instance = val
end

1

In [28]:
EvalTarget.class_class_instance

1

In [29]:
class EvalTarget
    attr_reader :instance_val
    
    def initialize
        @instance_val = 0
    end
end

:initialize

In [30]:
e1 = EvalTarget.new
e1.instance_val

0

In [31]:
e1.instance_eval do
    @instance_val = 10
end

10

In [32]:
e1.instance_val

10

In [33]:
class EvalTarget
    attr_reader :instance_val
    private
    def private_method
        @instance_val = 100
    end
end

:private_method

In [34]:
e1 = EvalTarget.new

#<EvalTarget:0x0000555f46c09b08 @instance_val=0>

In [35]:
e1.instance_eval do
    private_method
end
e1.instance_val

100

In [36]:
class EvalTarget; end

In [37]:
t1 = EvalTarget.new
t2 = EvalTarget.new

#<EvalTarget:0x0000555f46774188 @instance_val=0>

In [38]:
EvalTarget.class_eval do
    def class_method
        :class_method
    end
end

:class_method

In [39]:
p t1.class_method
t2.class_method

:class_method


:class_method

In [40]:
t1.instance_eval do
    def unique_method
        :unique_method
    end
end

:unique_method

In [41]:
p t1.unique_method
t2.unique_method

:unique_method


NoMethodError: undefined method `unique_method' for #<EvalTarget:0x0000555f46774188 @instance_val=0>

### 7-4-5 module_exec / class_exec / instance_exec

In [42]:
class EvalTarget
    CONST_VAL = "EvalTarget::CONST"
    attr_accessor :instance_val
end
CONST_VAL = 'CONST'
e1 = EvalTarget.new
e1.instance_eval {@instance_val = CONST_VAL}
# パーフェクトRubyだと"EvalTarget::CONST"になると書いてあるが"CONST"が返ってくる.
e1.instance_val

"CONST"

Ruby 1.9.3で試しても同じ結果になった.